In [2]:
class journey:
    def __init__(self, start, stop, no_of_people):
        self.start = start
        self.stop = stop
        self.no_of_people = no_of_people

class stop:
    def __init__(self, id):
        self.id = id
        self.next_stops = []
        self.rise = 0
        self.fall = 0
    
    def add_next_stop(self, stop_object):
        self.next_stops.extend(stop_object)

class bus:
    def __init__(self, first, last, route):
        self.first = first
        self.last = last
        self.route = route

In [3]:
stoplist = []

#Bus routing
for i in range(11):
    stoplist.append(stop(i))

#now connect the routes
stoplist[0].add_next_stop([stoplist[2]])
stoplist[1].add_next_stop([stoplist[2]])
stoplist[2].add_next_stop([stoplist[3]])
stoplist[4].add_next_stop([stoplist[3]])
stoplist[5].add_next_stop([stoplist[4]])
stoplist[6].add_next_stop([stoplist[3]])
stoplist[3].add_next_stop([stoplist[7]])
stoplist[7].add_next_stop([stoplist[8], stoplist[9], stoplist[10]])

In [4]:
# import random

# # Define the dictionary
# my_dict = {
#     8: [0, 1, 2, 3, 4, 5, 6, 7],
#     9: [0, 1, 2, 3, 4, 5, 6, 7],
#     10: [0, 1, 2, 3, 4, 5, 6, 7],
#     7: [0, 1, 2, 3, 4, 5, 6],
#     3: [0, 1, 2, 4, 5, 6],
#     2: [0, 1],
#     4: [5]
# }

# # Function to randomly select a key and a random value from its list
# def get_random_key_and_value(dictionary):
#     random_key = random.choice(list(dictionary.keys()))
#     random_value = random.choice(dictionary[random_key])
#     return random_key, random_value

# BookingList = []

# # Run the function 100 times
# for _ in range(5000):
#     key, value = get_random_key_and_value(my_dict)
#     BookingList.append([stoplist[value], stoplist[key]])
#     # print(f"Random Key: {key}, Random Value: {value}")

here is the code for the csv read

In [ ]:

import pandas as pd
df = pd.read_csv("file.csv")
BookingList = []
for i in df:
    BookingList.append(list(i))

In [5]:
##find the routes and update the bookinglist
import copy
from collections import deque


def bfs_shortest_path(start_stop, end_stop):
    # Initialize the queue with the starting stop and the path containing only the starting stop
    queue = deque([(start_stop, [start_stop])])
    visited = set()  # To keep track of visited stops

    while queue:
        current_stop, path = queue.popleft()

        if current_stop == end_stop:
            return [stop.id for stop in path]

        if current_stop not in visited:
            visited.add(current_stop)

            for next_stop in current_stop.next_stops:
                if next_stop not in visited:
                    new_path = path + [next_stop]
                    queue.append((next_stop, new_path))

    return None  # If no path is found

for i in range(len(BookingList)):
    BookingList[i] = bfs_shortest_path(BookingList[i][0], BookingList[i][1])
    for j in range(len(BookingList[i])):
        BookingList[i][j] = copy.deepcopy(stoplist[BookingList[i][j]])

In [6]:
##add start time to the lists


In [7]:
#based on start time append the lists to several shift modules(total 33 shifts between, 4 per hour + 1 extra at the end)

shifts = []
for i in range(33):
    shifts.append([])

for i in BookingList:
    shift = int((i[-1]-my_dict[i[0].id][-1]) / 25)
    shifts[shift].append(i[:-1])

In [8]:
def get_time(val):
    hr = str(int(val/100))
    min = val%100
    if( min == 25):
        min = "15"
    elif(min==50):
        min ="30"
    elif(min==75):
        min = "45"
    elif(min==0):
        min = "00"
    return str(hr)+":"+str(min)

In [12]:
#from here, do it for all shifts
import math
for o in range(len(shifts)):
    rise = []
    fall = []
    print(">>>>> for Shift ",o," <<<<<")
    for i in shifts[o]:
        
        rise.extend(i[:-1])
        fall.extend(i[1:])

    #now, see how many there will be rising and falling at each position
    for i in stoplist:
        i.rise = len([j for j in rise if j.id == i.id])
        i.fall = len([j for j in fall if j.id == i.id])
    bus_threshold = 1 #set according to need

    no_of_bus_at_stop = []
    bus_must_drop_to_this_loc = [0 for _ in range(len(stoplist))]
    number_at_bus = 0
    for i in stoplist:
        number_at_bus += i.rise - i.fall
        no_of_bus_at_stop.append(math.ceil((number_at_bus) / bus_threshold))
        bus_must_drop_to_this_loc[i.id] = i.fall

    for i in range(8):
        print("No of people in bus at location", i, ": ", no_of_bus_at_stop[i])

    for i in range(8, 11):
        print("for last stopage",i,"the number of people will fall is ", bus_must_drop_to_this_loc[i])
    bus_threshold = 30
    busseslist = []
    #route 0 to 2
    print(math.ceil(no_of_bus_at_stop[0]/bus_threshold), "no of buses from 0 to 2 at time", get_time(800+(o*25)))
    busseslist.append(math.ceil(no_of_bus_at_stop[0]/bus_threshold))
    #route 1 to 2
    print(math.ceil(no_of_bus_at_stop[1]/bus_threshold), "no of buses from 1 to 2 at time", get_time(800+(o*25)))
    busseslist.append(math.ceil(no_of_bus_at_stop[1]/bus_threshold))
    #route 2 to 3
    print(math.ceil(no_of_bus_at_stop[2]/bus_threshold), "no of buses from 2 to 3 at time", get_time(825+(o*25)))
    busseslist.append(math.ceil(no_of_bus_at_stop[0]/bus_threshold))
    #route 5 to 3
    print(math.ceil(max(no_of_bus_at_stop[5], no_of_bus_at_stop[4])/bus_threshold), "no of buses from 5 to 3 at time", get_time(800+(o*25)))
    busseslist.append(math.ceil(max(no_of_bus_at_stop[5], no_of_bus_at_stop[4])/bus_threshold))
    #route 6 to 3
    print(math.ceil(no_of_bus_at_stop[6]/bus_threshold), "no of buses from 6 to 3 at time", get_time(825+(o*25)))
    busseslist.append(math.ceil(no_of_bus_at_stop[6]/bus_threshold))
    #route 3 to 7
    print(math.ceil(no_of_bus_at_stop[3]/bus_threshold), "no of buses from 3 to 7 at time", get_time(850+(o*25)))
    busseslist.append(math.ceil(no_of_bus_at_stop[3]/bus_threshold))
    #route 7 to 8
    print(math.ceil(bus_must_drop_to_this_loc[8]/bus_threshold), "no of buses from 7 to 8 at time", get_time(875+(o*25)))
    busseslist.append(math.ceil(bus_must_drop_to_this_loc[8]/bus_threshold))
    #route 7 to 9
    print(math.ceil(bus_must_drop_to_this_loc[9]/bus_threshold), "no of buses from 7 to 9 at time", get_time(875+(o*25)))
    busseslist.append(math.ceil(bus_must_drop_to_this_loc[9]/bus_threshold))
    #route 7 to 10
    print(math.ceil(bus_must_drop_to_this_loc[10]/bus_threshold), "no of buses from 7 to 10 at time", get_time(875+(o*25)))
    busseslist.append(math.ceil(bus_must_drop_to_this_loc[10]/bus_threshold))
    

>>>>> for Shift  0  <<<<<
No of people in bus at location 0 :  23
No of people in bus at location 1 :  43
No of people in bus at location 2 :  40
No of people in bus at location 3 :  23
No of people in bus at location 4 :  19
No of people in bus at location 5 :  59
No of people in bus at location 6 :  74
No of people in bus at location 7 :  54
for last stopage 8 the number of people will fall is  18
for last stopage 9 the number of people will fall is  18
for last stopage 10 the number of people will fall is  18
1 no of buses from 0 to 2 at time 8:00
2 no of buses from 1 to 2 at time 8:00
2 no of buses from 2 to 3 at time 8:15
2 no of buses from 5 to 3 at time 8:00
3 no of buses from 6 to 3 at time 8:15
1 no of buses from 3 to 7 at time 8:30
1 no of buses from 7 to 8 at time 8:45
1 no of buses from 7 to 9 at time 8:45
1 no of buses from 7 to 10 at time 8:45
>>>>> for Shift  1  <<<<<
No of people in bus at location 0 :  26
No of people in bus at location 1 :  56
No of people in bus at l